<a href="https://colab.research.google.com/github/CattoYT/ValorantCommsBot/blob/rewrite/Valorant_Bot_LLM_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install diskcache flask_ngrok2 pyngrok

# DO NOT SET THIS TO EXTRA INDEX URL, YOU WILL MESS STUFF UP BADLY WIKTH PERFORAMNCE
!pip install -U llama-cpp-python --index-url https://abetlen.github.io/llama-cpp-python/whl/cu122


# a little code is taken from a previous project
from llama_cpp import Llama
class llamaLLM:
    def __init__(self, systemprompt, model="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF", filename="Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf"):


        self.messages = [
            # Sample data fed because I couldn't think of a way to get it to say the right shit
          {"role": "system", "content": systemprompt},
          {"role": "user", "content": "(party) teammate: clove hit 70"},
          {"role": "assistant", "content": "[HEALTHINDICATOR] Clove -70"},
          {"role": "user", "content": "(party) teammate: brim -140"},
          {"role": "assistant", "content": "[HEALTHINDICATOR] Brimstone -140"},
        ]

        self.llm = Llama.from_pretrained(
            repo_id=model,
            filename=filename,
            verbose=False,
            n_gpu_layers=-1,
            n_ctx=4096
        )
    def addToConversation(self, content, role):
        self.messages.append(
              {
                  "role": role,
                  "content": content
              }
          )
    def generateConversationResponse(self, message, person="user"):
        self.addToConversation(message, person) # it can also be system
        response = self.llm.create_chat_completion(self.messages)["choices"][0]['message']['content'] # forgive me
        self.addToConversation(response, "assistant")
        return response # no genuinely what the fuck is this

llm = llamaLLM("""
You are a gamer playing Valorant, and are talking in the in-game chat. Your teammates may callout damage to certain agents like 'jett 40' or 'sage -120'.
If this sent, you must respond in the format of "[HEALTHINDICATOR] {Agent} {Damange}", where agent, (Any valorant agent) and damage are replaced with the agent and the amount of damage,
but [HEALTHINDICATOR] must be as it is in front. It should not be replaced in any way. The damage should not accumulate. Only output the new damage value and not the total damage

There may be other words such as hit or damaged or any other word like that to reference damage.
Most other information should be removed if it doesn't fit this format
When the message you need to respond to is not a form of callout, in location or damage, you can respond naturally. Use informal speech, no capital letters, and little punctuation except where necessary, such as commas to separate different clauses.

In cases where the teammate is calling out status, and no extra callouts need to be given, do not give extra callouts. In caess where the teammate is agreeing with you, you should not respond.
When at least 2 people say gg, or ggwp, respond with ggwp


The following are valid Valorant Agents:
brimstone phoenix sage sova viper cypher reyna killjoy breach omen jett raze skye yoru astra kayo chamber neon fade harbor gekko deadlock iso clove
There may be abbreviations used for each agent, such as KJ for killjoy or DL for deadlock.
""")

#while True:
#    print()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
Looking in indexes: https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 MB 4.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [2]:
from flask import Flask, request, jsonify
from pyngrok import conf, ngrok
from google.colab import userdata
import threading
import json

app = Flask(__name__)
conf.get_default().auth_token = userdata.get('NGROKToken')


public_url = ngrok.connect("80", domain="cheaply-caring-pup.ngrok-free.app").public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, "80"))

app.config["BASE_URL"] = public_url

# basic, will improve later
# may need to change this later if its not needed
class ValorantChat:
    def __init__(self, channel, user, line):
        self.channel = channel
        self.user = user
        self.line = line

    def Empty(self):
        self.channel = ""
        self.user = ""
        self.line = ""

    def json(self):
        return json.dumps(
            {
                "channel": self.channel,
                "user": self.user,
                "line": self.line
            }
        )

    def raw(self):
        return f"({self.channel}) {self.user}{self.line}"

@app.route('/', methods=['POST'])
def processChat():
    chatmsg = json.loads(request.json)

    chatmsg = ValorantChat(chatmsg["channel"], chatmsg["user"], chatmsg["line"])

    print(chatmsg.raw())
    response = llm.generateConversationResponse(chatmsg.raw)
    print(response + "\n")
    return response
    pass

def start():
    app.run(host='0.0.0.0', port=80)

start()

 * ngrok tunnel "https://cheaply-caring-pup.ngrok-free.app" -> "http://127.0.0.1:80"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:80
 * Running on http://172.28.0.12:80
INFO:werkzeug:Press CTRL+C to quit


(All) Ansah: ez


INFO:werkzeug:127.0.0.1 - - [10/Aug/2024 12:25:18] "POST / HTTP/1.1" 200 -


idk what's going on



INFO:werkzeug:127.0.0.1 - - [10/Aug/2024 12:25:21] "POST / HTTP/1.1" 200 -


(All) Me: gg
same here

